In [7]:
# 把scLLM的位置添加进system path保证可以import scLLM
import sys
root = "/home/shi/WorkSpace/projects/scMultiNet_Data/"
code_loc = "/home/shi/WorkSpace/projects/scMultiNet_workspace/"
sys.path.append(code_loc)
# 数据集读取
import dill
raw_data_loc = root + "/Step_1_data/Dataset_cook/TrVal_dataset_PC_TGFb_GTlabel5.pkl"
# 用dill打开loc0的pkl 文件读取dataset
with open(raw_data_loc,"rb") as f:
    [trainset,valset,_,label_dict] = dill.load(f)
# 输出数据集信息
print("trainset size: ",len(trainset))
print("valset size: ",len(valset)) if valset is not None else print("no valset")
print(label_dict)

trainset size:  5881
valset size:  1471
{'3d': 0, '7d': 1, '1d': 2, '8h': 3, '0d': 4}


In [8]:
# vocab 读取 
vocab_loc = code_loc + "/Experiment/support_data/vocab_16k.json"
import json
with open(vocab_loc,"r") as f:
    vocab = json.load(f)


# 根据字典的值进行排序，并获取排序后的键
sorted_genes = sorted(vocab, key=vocab.get)

# 现在 sorted_genes 是一个按值排序的基因名称列表
sorted_genes.append("CLS")
print(sorted_genes[:20])
print(len(sorted_genes))


['A1BG', 'A1CF', 'A2M', 'A2ML1', 'A4GALT', 'A4GNT', 'AAAS', 'AACS', 'AADAC', 'AADACL2', 'AADAT', 'AAGAB', 'AAK1', 'AAMDC', 'AAMP', 'AANAT', 'AAR2', 'AARS2', 'AARSD1', 'AASDH']
16907


In [9]:
import numpy as np

# 步骤 0: 数据位置
data_locs = [
    root+"/Step_6_data/data/multi_cls0.pkl",
    root+"/Step_6_data/data/multi_cls1.pkl",
    root+"/Step_6_data/data/multi_cls2.pkl",
    root+"/Step_6_data/data/multi_cls3.pkl",
    root+"/Step_6_data/data/multi_cls4.pkl",
]
# 步骤 1: 加载数据
# 文件保存在pkl
import pickle
def load_data(file_path):
    with open(file_path, 'rb') as f:
        [pred_list,label_list,weights_list,weighted_feature_list] = pickle.load(f)
    print(pred_list.shape)
    print(label_list.shape)
    print(weights_list.shape)
    print(weighted_feature_list.shape)
    return pred_list,label_list,weights_list,weighted_feature_list



In [10]:
pred_lists = []
weights_lists = []
weighted_feature_lists = []
for l in data_locs:
    pred_list,label_list,weights_list,weighted_feature_list = load_data(l)
    pred_lists.append(pred_list)
    weights_lists.append(weights_list)
    weighted_feature_lists.append(weighted_feature_list)

(1471, 2)
(1471, 1)
(1471, 16907)
(1471, 16907)
(1471, 2)
(1471, 1)
(1471, 16907)
(1471, 16907)
(1471, 2)
(1471, 1)
(1471, 16907)
(1471, 16907)
(1471, 2)
(1471, 1)
(1471, 16907)
(1471, 16907)
(1471, 2)
(1471, 1)
(1471, 16907)
(1471, 16907)


In [11]:
# weights_lists 是一个包含5个模型 weights_list 的列表
# 初始化 model_weights
model_weights = np.zeros([weights_lists[0].shape[0] , weights_lists[0].shape[1]])
# 初始化 avg_weights
avg_0 = [np.zeros([weights_lists[0].shape[1],1]),0]
avg_1 = [np.zeros([weights_lists[0].shape[1],1]),0]
avg_2 = [np.zeros([weights_lists[0].shape[1],1]),0]
avg_3 = [np.zeros([weights_lists[0].shape[1],1]),0]
avg_4 = [np.zeros([weights_lists[0].shape[1],1]),0]
# 遍历每个样本
for i in range(weights_lists[0].shape[0]):
    # 获取当前样本的真实类别
    label = int(label_list[i])

    # 选择并复制对应类别的 weights_list
    model_weights[i, :] = weights_lists[label][i, :]
    # 更新 avg_weights
    if label == 0:
        avg_0[0] += weights_lists[label][i, :].reshape(-1,1)
        avg_0[1] += 1
    elif label == 1:
        avg_1[0] += weights_lists[label][i, :].reshape(-1,1)
        avg_1[1] += 1
    elif label == 2:
        avg_2[0] += weights_lists[label][i, :].reshape(-1,1)
        avg_2[1] += 1
    elif label == 3:
        avg_3[0] += weights_lists[label][i, :].reshape(-1,1)
        avg_3[1] += 1
    elif label == 4:
        avg_4[0] += weights_lists[label][i, :].reshape(-1,1)
        avg_4[1] += 1

# 计算平均权重
avg_0 = avg_0[0] / avg_0[1]
avg_1 = avg_1[0] / avg_1[1]
avg_2 = avg_2[0] / avg_2[1]
avg_3 = avg_3[0] / avg_3[1]
avg_4 = avg_4[0] / avg_4[1]

rev_label_dict = {v:k for k,v in label_dict.items()}
dict_score = {
    rev_label_dict[0]:avg_0.flatten(),
    rev_label_dict[1]:avg_1.flatten(),
    rev_label_dict[2]:avg_2.flatten(),
    rev_label_dict[3]:avg_3.flatten(),
    rev_label_dict[4]:avg_4.flatten(),
}


In [12]:
import pandas as pd
# 从dict_score生成pandas dataframe，每行的index是基因名称sorted_genes
df = pd.DataFrame.from_dict(dict_score, orient='index', columns=sorted_genes)
df = df.T
df.to_csv(root+"/Step_6_data/avg_attention_only.csv")

In [13]:
# weights_lists 是一个包含5个模型 weights_list 的列表
# 初始化 model_weights
model_weight_features = np.zeros([weighted_feature_lists[0].shape[0] , weighted_feature_lists[0].shape[1]])
# 初始化 avg_weights
avg_0 = [np.zeros([weighted_feature_lists[0].shape[1],1]),0]
avg_1 = [np.zeros([weighted_feature_lists[0].shape[1],1]),0]
avg_2 = [np.zeros([weighted_feature_lists[0].shape[1],1]),0]
avg_3 = [np.zeros([weighted_feature_lists[0].shape[1],1]),0]
avg_4 = [np.zeros([weighted_feature_lists[0].shape[1],1]),0]
# 遍历每个样本
for i in range(weighted_feature_lists[0].shape[0]):
    # 获取当前样本的真实类别
    label = int(label_list[i])

    # 选择并复制对应类别的 weights_list
    model_weight_features[i, :] = weighted_feature_lists[label][i, :]
    # 更新 avg_weights
    if label == 0:
        avg_0[0] += weighted_feature_lists[label][i, :].reshape(-1,1)
        avg_0[1] += 1
    elif label == 1:
        avg_1[0] += weighted_feature_lists[label][i, :].reshape(-1,1)
        avg_1[1] += 1
    elif label == 2:
        avg_2[0] += weighted_feature_lists[label][i, :].reshape(-1,1)
        avg_2[1] += 1
    elif label == 3:
        avg_3[0] += weighted_feature_lists[label][i, :].reshape(-1,1)
        avg_3[1] += 1
    elif label == 4:
        avg_4[0] += weighted_feature_lists[label][i, :].reshape(-1,1)
        avg_4[1] += 1

# 计算平均权重
avg_0 = avg_0[0] / avg_0[1]
avg_1 = avg_1[0] / avg_1[1]
avg_2 = avg_2[0] / avg_2[1]
avg_3 = avg_3[0] / avg_3[1]
avg_4 = avg_4[0] / avg_4[1]

rev_label_dict = {v:k for k,v in label_dict.items()}
dict_score = {
    rev_label_dict[0]:avg_0.flatten(),
    rev_label_dict[1]:avg_1.flatten(),
    rev_label_dict[2]:avg_2.flatten(),
    rev_label_dict[3]:avg_3.flatten(),
    rev_label_dict[4]:avg_4.flatten(),
}

In [14]:
# 从dict_score生成pandas dataframe，每行的index是基因名称sorted_genes
df = pd.DataFrame.from_dict(dict_score, orient='index', columns=sorted_genes)
df = df.T
df.to_csv(root+"/Step_6_data/avg_attention_x_gene_expression.csv")

In [9]:
import pandas as pd

df = pd.read_csv(root+"/Step_6_data/avg_weight_features.csv")
# 重命名第一列
df = df.rename(columns={"Unnamed: 0":"gene"})
df = df.set_index("gene")

# 获取每个类别排名前10的基因
top_10_genes = {}
for label in ["0d","8h","1d","3d","7d"]:
    top_10_genes[label] = df[label].sort_values(ascending=False).index[:50].tolist()


In [10]:
pd.DataFrame(top_10_genes).to_csv(root+"/Step_6_data/top_50_genes.csv")